- 용인시 행정동 2개에 대한 set-covering optimization을 통해 최적의 입지 갯수 p개 설정 
- 기준은 Set-covering모형은 수요지와 공공시설 간에 최대 허용 거리(450m)를 미리 정해놓고, 그 조건을 만족시키는 범위 내에서 설치할 전기차 충전소 입지의 개수를 최소로 하는 것이 목적 (고객의 접근성 요소 고려)


In [28]:
import pandas as pd
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB


### Input Data

In [29]:
path = 'C:/bigcon'
sungbok_od = pd.read_csv(path + 'output\data\checkpoint\sungbok_od_mat.csv', encoding = 'utf-8')

In [31]:
# demand 따로 추출 
sungbok_demand = sungbok_od[['앱실행수']]
sungbok_demand

,앱실행수
0,12461.306
1,9266.882
2,8375.573
3,7751.544
4,7289.718
5,7020.950
6,6897.198
7,6018.023
8,5815.623
9,5663.007


In [32]:
sungbok_od_drop = sungbok_od.drop(['앱실행수'],axis = 1)
sungbok_od_drop

,Unnamed: 0,수지제일아이조움,성복역롯데캐슬파크나인1차,성복역 롯데캐슬 골드타운,성동마을 LG빌리지2차,성복역 롯데캐슬 클라시엘(파크나인2차),성동마을 강남빌리지,성남마을 벽산첼시빌2,버들치마을 성복 힐스테이트 1차,성복역KCC스위첸,...,성동마을 LG빌리지3차,버들치마을 성복자이2차,성동마을 수지자이2차,성동마을 LG빌리지 6차,버들치마을 성복 힐스테이트 3차,성동마을 수지자이아파트,용인 성복아이파크,아이비힐,수지성복 효성해링턴코트,성복동
0,0,255.085827,863.400778,156.868057,697.836335,858.768259,598.699495,1600.044064,1222.307134,957.851414,...,1205.882509,2067.531557,1580.500066,1314.761257,2029.243136,1601.924708,2184.940084,2436.638371,3088.234835,1357.777840
1,1,1116.791230,799.111957,1161.837987,587.463362,428.224335,522.583659,775.827337,474.263553,314.664826,...,125.190510,1052.701371,521.630965,234.138167,968.873663,530.295915,1111.335124,1372.934161,2018.718290,413.864827
2,2,880.960418,723.014250,911.607962,454.048082,376.642854,313.824397,970.768729,611.697414,340.854746,...,392.688004,1312.425291,788.471035,501.085651,1236.997585,782.189505,1379.809528,1641.545103,2287.323987,635.718905
3,3,216.110824,718.688655,204.828691,548.839169,713.130594,460.318254,1452.352749,1075.977234,816.415979,...,1084.371841,1927.413744,1449.130753,1192.684876,1896.121462,1486.993918,2055.526488,2303.354538,2955.895737,1217.567480
4,4,2142.879439,1514.399531,2230.481384,1523.923233,1342.439370,1596.892367,660.588574,988.594193,1237.178565,...,1091.411002,156.570461,694.644890,1000.945327,366.341427,900.297188,451.651289,432.962501,998.929794,839.311536
5,5,1365.225727,948.239389,1419.266518,793.070999,607.974765,769.890491,636.698719,469.085413,475.928206,...,147.310123,798.417469,258.164165,54.906249,700.690093,308.043436,842.531516,1103.873515,1749.660120,287.579266
6,6,1438.531355,793.792728,1535.296408,826.150238,661.214943,934.170667,86.277006,298.373034,584.775549,...,645.037146,633.483486,482.008406,619.410011,720.781699,774.522335,910.630586,1074.672550,1704.475300,291.611451
7,7,528.255503,771.729892,512.392466,511.855817,603.247994,330.843685,1334.241179,951.869966,664.119155,...,839.119580,1742.341506,1230.357072,947.614433,1680.172220,1222.745783,1825.965327,2085.979654,2733.211305,1042.593890
8,8,1759.950531,1080.868254,1864.032972,1160.333611,1006.509202,1281.857932,266.430583,648.024485,936.751774,...,962.331756,506.891045,684.435886,913.487110,694.231247,981.387869,864.547450,938.351188,1507.238322,615.997799
9,9,1080.890323,749.228405,1130.036490,540.834311,378.738079,485.253445,753.361716,436.636301,264.689065,...,157.092312,1058.219501,535.353953,263.573950,984.944849,567.669004,1132.823658,1390.781554,2039.023806,397.711457


In [33]:
# 출발지 기호 변경
depart_oldname = sungbok_od_drop['Unnamed: 0'].unique().tolist()
depart_newname = [] # 바뀌는 값
for i in range(1,len(depart_oldname)+1):
    depart_newname.append('D' + str(i))

sungbok_od_drop['수요지'] = sungbok_od_drop['Unnamed: 0'].replace(depart_oldname,depart_newname) # 수요지 열 재생성
sungbok_od_drop = sungbok_od_drop.set_index(['수요지'])
sungbok_od_drop.drop(['Unnamed: 0'],axis=1, inplace=True)

# 도착지 기호 변경
arrive_oldname = sungbok_od_drop.columns.unique().tolist()
arrive_newname = [] # 바뀌는 값
for i in range(1,len(arrive_oldname)+1):
    arrive_newname.append('A' + str(i))

sungbok_od_drop.columns = arrive_newname
sungbok_od_drop

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A17,A18,A19,A20,A21,A22,A23,A24,A25,A26
수요지,,,,,,,,,,,,,,,,,,,,,
D1,255.085827,863.400778,156.868057,697.836335,858.768259,598.699495,1600.044064,1222.307134,957.851414,1114.441843,...,1205.882509,2067.531557,1580.500066,1314.761257,2029.243136,1601.924708,2184.940084,2436.638371,3088.234835,1357.777840
D2,1116.791230,799.111957,1161.837987,587.463362,428.224335,522.583659,775.827337,474.263553,314.664826,309.329679,...,125.190510,1052.701371,521.630965,234.138167,968.873663,530.295915,1111.335124,1372.934161,2018.718290,413.864827
D3,880.960418,723.014250,911.607962,454.048082,376.642854,313.824397,970.768729,611.697414,340.854746,443.802087,...,392.688004,1312.425291,788.471035,501.085651,1236.997585,782.189505,1379.809528,1641.545103,2287.323987,635.718905
D4,216.110824,718.688655,204.828691,548.839169,713.130594,460.318254,1452.352749,1075.977234,816.415979,973.044521,...,1084.371841,1927.413744,1449.130753,1192.684876,1896.121462,1486.993918,2055.526488,2303.354538,2955.895737,1217.567480
D5,2142.879439,1514.399531,2230.481384,1523.923233,1342.439370,1596.892367,660.588574,988.594193,1237.178565,1080.644711,...,1091.411002,156.570461,694.644890,1000.945327,366.341427,900.297188,451.651289,432.962501,998.929794,839.311536
D6,1365.225727,948.239389,1419.266518,793.070999,607.974765,769.890491,636.698719,469.085413,475.928206,372.712966,...,147.310123,798.417469,258.164165,54.906249,700.690093,308.043436,842.531516,1103.873515,1749.660120,287.579266
D7,1438.531355,793.792728,1535.296408,826.150238,661.214943,934.170667,86.277006,298.373034,584.775549,444.640266,...,645.037146,633.483486,482.008406,619.410011,720.781699,774.522335,910.630586,1074.672550,1704.475300,291.611451
D8,528.255503,771.729892,512.392466,511.855817,603.247994,330.843685,1334.241179,951.869966,664.119155,812.752679,...,839.119580,1742.341506,1230.357072,947.614433,1680.172220,1222.745783,1825.965327,2085.979654,2733.211305,1042.593890
D9,1759.950531,1080.868254,1864.032972,1160.333611,1006.509202,1281.857932,266.430583,648.024485,936.751774,796.983470,...,962.331756,506.891045,684.435886,913.487110,694.231247,981.387869,864.547450,938.351188,1507.238322,615.997799


In [36]:
# 성복동 arc 및 demand(P/T) dict
dic = {}

for i in sungbok_od_drop.index:
    for name in sungbok_od_drop.columns:
        key = (i, name)
        value = sungbok_od_drop.loc[i, name]
        dic[key] = value

In [37]:
# 수요지와 도착지 
demand = {i for (i,j) in dic.keys()}
print("수요지 =",demand)

arrival = {j for (i,j) in dic.keys()}
print("전기차 충전소 후보지 =",arrival)

수요지 = {'D13', 'D15', 'D16', 'D3', 'D18', 'D8', 'D11', 'D17', 'D9', 'D6', 'D10', 'D2', 'D5', 'D4', 'D19', 'D20', 'D12', 'D7', 'D1', 'D14'}
전기차 충전소 후보지 = {'A4', 'A15', 'A19', 'A7', 'A3', 'A23', 'A2', 'A25', 'A13', 'A6', 'A16', 'A12', 'A26', 'A14', 'A18', 'A17', 'A5', 'A10', 'A8', 'A22', 'A24', 'A20', 'A21', 'A1', 'A11', 'A9'}


In [38]:
# 450m 이내에 있는 도착지 set 
close_arrival = { j : list() for j in demand}

for i in demand:
    for j in arrival:
        if dic[i,j] <= 450:
            close_arrival[i].append(j)
    print("수요지",i,"기준 450m 이내에 있는 후보지는",close_arrival[i])

수요지 D13 기준 450m 이내에 있는 후보지는 ['A4', 'A2', 'A6', 'A5', 'A1']
수요지 D15 기준 450m 이내에 있는 후보지는 ['A4', 'A6', 'A26', 'A14', 'A17', 'A5', 'A10', 'A8', 'A20', 'A9']
수요지 D16 기준 450m 이내에 있는 후보지는 ['A15', 'A19', 'A16', 'A26', 'A17', 'A22', 'A20']
수요지 D3 기준 450m 이내에 있는 후보지는 ['A6', 'A14', 'A17', 'A5', 'A10', 'A11', 'A9']
수요지 D18 기준 450m 이내에 있는 후보지는 ['A15', 'A19', 'A7', 'A13', 'A16', 'A12', 'A26', 'A18']
수요지 D8 기준 450m 이내에 있는 후보지는 ['A6', 'A11']
수요지 D11 기준 450m 이내에 있는 후보지는 ['A4', 'A2', 'A6', 'A14', 'A5', 'A10', 'A8', 'A9']
수요지 D17 기준 450m 이내에 있는 후보지는 ['A15', 'A19', 'A23', 'A16', 'A18', 'A21']
수요지 D9 기준 450m 이내에 있는 후보지는 ['A15', 'A7', 'A13', 'A12']
수요지 D6 기준 450m 이내에 있는 후보지는 ['A19', 'A26', 'A17', 'A10', 'A22', 'A20']
수요지 D10 기준 450m 이내에 있는 후보지는 ['A26', 'A14', 'A17', 'A5', 'A10', 'A8', 'A20', 'A9']
수요지 D2 기준 450m 이내에 있는 후보지는 ['A26', 'A14', 'A17', 'A5', 'A10', 'A20', 'A9']
수요지 D5 기준 450m 이내에 있는 후보지는 ['A13', 'A16', 'A12', 'A18', 'A24', 'A21']
수요지 D4 기준 450m 이내에 있는 후보지는 ['A3', 'A1', 'A11']
수요지 D19 기준 450m 이내에 있

### Model deployment

In [39]:
model2 = gp.Model("electric_set_cover_v2")

# variables
x = model2.addVars(arrival, vtype = GRB.BINARY, name='x_j')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-03


In [40]:
# objective function
obj = gp.quicksum(x[j] for j in arrival)
        
model2.setObjective(obj, GRB.MINIMIZE)

In [41]:
# constraints

model2.addConstrs(gp.quicksum(x[j] for j in close_arrival[i]) >= 1 for i in demand)


{'D13': <gurobi.Constr *Awaiting Model Update*>,
 'D15': <gurobi.Constr *Awaiting Model Update*>,
 'D16': <gurobi.Constr *Awaiting Model Update*>,
 'D3': <gurobi.Constr *Awaiting Model Update*>,
 'D18': <gurobi.Constr *Awaiting Model Update*>,
 'D8': <gurobi.Constr *Awaiting Model Update*>,
 'D11': <gurobi.Constr *Awaiting Model Update*>,
 'D17': <gurobi.Constr *Awaiting Model Update*>,
 'D9': <gurobi.Constr *Awaiting Model Update*>,
 'D6': <gurobi.Constr *Awaiting Model Update*>,
 'D10': <gurobi.Constr *Awaiting Model Update*>,
 'D2': <gurobi.Constr *Awaiting Model Update*>,
 'D5': <gurobi.Constr *Awaiting Model Update*>,
 'D4': <gurobi.Constr *Awaiting Model Update*>,
 'D19': <gurobi.Constr *Awaiting Model Update*>,
 'D20': <gurobi.Constr *Awaiting Model Update*>,
 'D12': <gurobi.Constr *Awaiting Model Update*>,
 'D7': <gurobi.Constr *Awaiting Model Update*>,
 'D1': <gurobi.Constr *Awaiting Model Update*>,
 'D14': <gurobi.Constr *Awaiting Model Update*>}

In [42]:
model2.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 20 rows, 26 columns and 115 nonzeros
Model fingerprint: 0xbf284751
Variable types: 0 continuous, 26 integer (26 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 20 rows and 26 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 5 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [43]:
for v in model2.getVars():
    if v.X != 0:
        print("%s : %f" % (v.Varname, v.X))

print(model2.objVal) # 최적의 입지 갯수 P = 5개

x_j[A7] : 1.000000
x_j[A6] : 1.000000
x_j[A26] : 1.000000
x_j[A18] : 1.000000
x_j[A1] : 1.000000
5.0
